In [ ]:
import RPi.GPIO as gpio
from flask import Flask

app = Flask(__name__)

# set pin mapping to BOARD
gpio.setmode(gpio.BCM)

# turn off channel warnings messages
gpio.setwarnings(False)

# Set GPIO pins as output
gpio.setup(18, gpio.OUT)
gpio.setup(15, gpio.OUT)

# set GPIO pins as inputs
gpio.setup(2, gpio.IN)  # GPIO 2 -> Left IR out
gpio.setup(3, gpio.IN)  # GPIO 3 -> Right IR out

# turn on left motor
def leftOn():
    gpio.output(15, 1)

# turn off left motor
def leftOff():
    gpio.output(15, 0)

# turn on right motor
def rightOn():
    gpio.output(18, 1)

# turn off right motor
def rightOff():
    gpio.output(18, 0)

# turn off all motors
def stopAll():
    gpio.output(18, 0)
    gpio.output(15, 0)

# Main program loop
def line_following():
    while True:
        # if left and right sensors are off, stop both motors
        if gpio.input(2) == 0 and gpio.input(3) == 0:
            leftOff()
            rightOff()

        # if both sensors are on, then turn both motors on
        if gpio.input(2) == 1 and gpio.input(3) == 1:
            leftOn()
            rightOn()

        # if left sensor is on, turn right motor off (pivot left)
        if gpio.input(2) == 1 and gpio.input(3) == 0:
            leftOn()
            rightOff()

        # if right sensor is on, turn left motor off (pivot right)
        if gpio.input(2) == 0 and gpio.input(3) == 1:
            leftOff()
            rightOn()

@app.route("/")
def index():
    return '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>PiBot Control</title>
    </head>
    <body>
        <h1>PiBot Control</h1>
        <form action="/on" method="get">
            <button type="submit">Turn On</button>
        </form>
        <form action="/off" method="get">
            <button type="submit">Turn Off</button>
        </form>
    </body>
    </html>
    '''

@app.route("/on")
def turn_on():
    line_following()
    return "PiBot turned on"

@app.route("/off")
def turn_off():
    stopAll()
    return "PiBot turned off"

if __name__ == '__main__':
    stopAll()  # make sure all pins are set to off
    app.run(host='0.0.0.0')

gpio.cleanup()